<a href="https://colab.research.google.com/github/cumason123/NCAA-NBA-Picker/blob/master/RNN_Player_Point_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

import tensorflow as tf
import pandas as pd
import numpy as np

#tf.enable_eager_execution()

!gsutil cp gs://stardust-hackathon/ncaa-mbb/player_box.csv /tmp/

df = pd.read_csv("/tmp/player_box.csv")

Authenticated
Copying gs://stardust-hackathon/ncaa-mbb/player_box.csv...
- [1 files][ 71.6 MiB/ 71.6 MiB]                                                
Operation completed over 1 objects/71.6 MiB.                                     


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
print(df.head(10))

   team_code  season             game_id   game_date  player_id jersey_num  \
0      768.0    2018    327-768-2019-1-9  2019-01-09  2082634.0          1   
1      768.0    2018   698-768-2019-1-15  2019-01-15  2082634.0          1   
2      768.0    2018  768-606-2018-11-18  2018-11-18  2082634.0          1   
3      768.0    2018  768-352-2018-12-30  2018-12-30  2082634.0          1   
4      768.0    2018   768-521-2019-1-12  2019-01-12  2082634.0          1   
5      768.0    2018    768-51-2019-1-21  2019-01-21  2082634.0          1   
6      768.0    2018  768-315-2018-12-22  2018-12-22  2082634.0          1   
7      768.0    2018    703-768-2019-1-5  2019-01-05  2082634.0          1   
8      768.0    2018   768-328-2019-1-19  2019-01-19  2082634.0          1   
9      768.0    2018    768-700-2019-1-2  2019-01-02  2082634.0          1   

    pts   fga  fga3  fgm     ...       ast  blk  stl  dreb  oreb   reb   pf  \
0  17.0  14.0   0.0  7.0     ...       0.0  1.0  0.0   5.0   7

In [0]:
player = df[df['player_id'] == 2082634.0]

In [0]:
player_info = player.loc[:, 'pts':'mins_played':1]

In [0]:
x = player_info[1:]

In [0]:
y = player_info.loc[:, 'pts'][:-1]

In [0]:
x = x.iloc[::-1]
y = y.iloc[::-1]

In [0]:
xtensor = tf.convert_to_tensor(x.values, dtype = tf.float32)
ytensor = tf.convert_to_tensor(y.values, dtype = tf.float32)

xtensor = tf.reshape(xtensor, (9, 1, 17))
ytensor = tf.reshape(ytensor, (9, 1, 1))

xtensor = tf.math.l2_normalize(
    xtensor,
    axis=(0,1)
)
ytensor = tf.math.l2_normalize(
    ytensor,
    axis=(0,1)
)


print(xtensor.shape, ytensor.shape)
print(xtensor)

(9, 1, 17) (9, 1, 1)
Tensor("l2_normalize:0", shape=(9, 1, 17), dtype=float32)


In [0]:
tf.reshape(y[0], (1,1))

<tf.Tensor 'Reshape_2:0' shape=(1, 1) dtype=float64>

In [0]:
lstm = tf.contrib.rnn.BasicLSTMCell(17)

initial_state = state = lstm.zero_state(1, dtype=tf.float32)

dense1 = tf.layers.Dense(units = 1)

loss = 0
for i in range(9):
    output, state = lstm(xtensor[i], state)
    logits = dense1(output)
    
    loss += tf.nn.sigmoid_cross_entropy_with_logits(labels=ytensor[i], logits=logits)    

final_state = state
print(loss)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Tensor("add_8:0", shape=(1, 1), dtype=float32)


In [0]:
import tensorflow as tf
from tensorflow.contrib import rnn


'''
To classify images using a recurrent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
'''

# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 1
display_step = 200

# Network Parameters
num_input = 17
timesteps = 9 # timesteps
num_hidden = 128 # hidden layer num of features
num_scores = 1 

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_scores])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_scores]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_scores]))
}


def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

logits = RNN(X, weights, biases)
prediction = tf.nn.sigmoid(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()



# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = (x.values,y.values)  ######mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        batch_y = batch_y.reshape((timesteps, num_scores))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 1
    test_data = x.values.reshape(1,9,17)
    test_label = y.values.reshape(9,1)
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= -228.8556, Training Accuracy= 1.000
Step 200, Minibatch Loss= -2235.3918, Training Accuracy= 1.000
Step 400, Minibatch Loss= -3865.2004, Training Accuracy= 1.000
Step 600, Minibatch Loss= -5516.9746, Training Accuracy= 1.000
Step 800, Minibatch Loss= -7200.4829, Training Accuracy= 1.000
Step 1000, Minibatch Loss= -8884.0547, Training Accuracy= 1.000
Step 1200, Minibatch Loss= -10567.6270, Training Accuracy= 1.000
Step 1400, Minibatch Loss= -12251.1836, Training Accuracy= 1.000
Step 1600, Minibatch Loss= -13934.7354, Training Accuracy= 1.000
Step 1800, Minibatch Loss= -15618.2988, Training Accuracy= 1.000
Step 2000, Minibatch Loss= -17301.9844, Training Accuracy= 1.000
Step 2200, Minibatch Loss= -18985.7012, Training Accuracy= 1.000
Step 2400, Minibatch Loss= -20669.4141, Training Accuracy= 1.000
Step 2600, Minibatch Loss= -22353.1250, Training Accuracy= 1.000
Step 2800, Minibatch Loss= -24036.8359, Training Accuracy= 1.000
Step 3000, Minibatch Loss= -25720.5527,

In [0]:
ytest = list(player_info.loc[:, 'pts'][:-1])
#print(ytest)
s = 0
alpha = 0.7
average = 0
for i in range(0,9-1):
    s = alpha * s + (1-alpha) * ytest[i]
    average += abs(s - ytest[i+1])/(max(ytest[i+1], s))
average /= 9
print('exponential average estimation accuracy')
print(average)

exponential average estimation accuracy
0.4940318001683852
